In [1]:
import pandas as pd
import numpy as np
import os 

In [2]:
dataset = pd.read_csv("TRADES_CopyTr_90D_ROI.csv")

In [3]:
dataset.head()

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


In [4]:
dataset.iloc[149,-1]

"[{'time': 1718977395000, 'symbol': 'BNBUSDT', 'side': 'SELL', 'price': 580.63, 'fee': -0.19973672, 'feeAsset': 'USDT', 'quantity': 998.6836, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'BNB', 'qty': 1.72, 'positionSide': 'BOTH', 'activeBuy': True}, {'time': 1718961376000, 'symbol': 'JASMYUSDT', 'side': 'BUY', 'price': 0.027267, 'fee': -0.19914453, 'feeAsset': 'USDT', 'quantity': 398.289069, 'quantityAsset': 'USDT', 'realizedProfit': 127.07864464, 'realizedProfitAsset': 'USDT', 'baseAsset': 'JASMY', 'qty': 14607.0, 'positionSide': 'BOTH', 'activeBuy': True}, {'time': 1718961274000, 'symbol': 'JASMYUSDT', 'side': 'BUY', 'price': 0.027319, 'fee': -0.00845523, 'feeAsset': 'USDT', 'quantity': 16.910461, 'quantityAsset': 'USDT', 'realizedProfit': 5.35301642, 'realizedProfitAsset': 'USDT', 'baseAsset': 'JASMY', 'qty': 619.0, 'positionSide': 'BOTH', 'activeBuy': True}, {'time': 1718961274000, 'symbol': 'JASMYUSDT', 'side': 'BUY', 'price': 0.0273

In [5]:
import ast
for i in range(150):
    # Extracting the name and data string
    name = dataset.iloc[i, 0]
    data_str = dataset.iloc[i, 1]

    # Converting the data string to a list of dictionaries
    if type(data_str) == str:
        data_list = ast.literal_eval(data_str)

    # Creating a DataFrame from the list of dictionaries
    data = pd.DataFrame(data_list)

    # Saving the DataFrame to a CSV file
    data.to_csv(r'C:\Users\pc\Desktop\PrimeTrade\Ports\{}.csv'.format(name), index=False)

In [6]:
lst = os.listdir(r"C:\Users\pc\Desktop\PrimeTrade\Ports")
k = []
for i in lst:
    dict = {}

    df = pd.read_csv(r"C:\Users\pc\Desktop\PrimeTrade\Ports\{}".format(i))

    # Calculate Total Realized Profit (PnL)
    total_realized_profit = df['realizedProfit'].sum()

    # Calculate Total Invested Capital (assumed as sum of prices * quantities)
    total_invested_capital = df['quantity'].sum()

    # Calculate ROI
    roi = (total_realized_profit / total_invested_capital) * 100

    # Calculate Daily Returns
    df['daily_return'] = df['realizedProfit'] / total_invested_capital
    average_daily_return = df['daily_return'].mean()
    std_daily_return = df['daily_return'].std()

    # Calculate Sharpe Ratio (assuming risk-free rate is 0)
    sharpe_ratio = average_daily_return / std_daily_return

    # Calculate Maximum Drawdown (MDD)
    cumulative_returns = (1 + df['daily_return']).cumprod()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()

    # Calculate Win Rate
    win_trades = df[df['realizedProfit'] > 0].shape[0]
    total_trades = df.shape[0]
    win_rate = (win_trades / total_trades) * 100

    # Number of Win Positions
    win_positions = win_trades

    # Total Positions
    total_positions = total_trades

    dict["Port"] = i[:-4]
    dict["Total Realized Profit (PnL)"] = total_realized_profit
    dict["ROI"] = roi
    dict["Sharpe Ratio"] = sharpe_ratio
    dict["Maximum Drawdown (MOD)"] = max_drawdown
    dict["Win Rate"] = win_rate
    dict['Win Positions'] = win_positions
    dict['Total Positions'] = total_positions

    k.append(dict)
df = pd.DataFrame(k)

C:\Users\pc\AppData\Local\Temp\ipykernel_22612\2182464596.py:23: RuntimeWarning: invalid value encountered in scalar divide
  sharpe_ratio = average_daily_return / std_daily_return


In [7]:
df.isnull().sum()

Port                           0
Total Realized Profit (PnL)    0
ROI                            0
Sharpe Ratio                   1
Maximum Drawdown (MOD)         0
Win Rate                       0
Win Positions                  0
Total Positions                0
dtype: int64

In [8]:
df.fillna(0,inplace=True)

In [9]:
df["score"] = df.apply(lambda x : x["Total Realized Profit (PnL)"] * 0.25 + x["ROI"] *0.25 + x["Sharpe Ratio"] * 0.25 + x["Maximum Drawdown (MOD)"] * 0.15 + x["Win Rate"] * 0.1 + x["Win Positions"] * 0.05 + x["Total Positions"] * 0.05,axis=1)
df = df.sort_values(by="score",ascending=False)
df["rank"] = df["score"].rank(ascending=False)

In [10]:
df.sort_values(by="rank",ignore_index=True).head(20)

,Port,Total Realized Profit (PnL),ROI,Sharpe Ratio,Maximum Drawdown (MOD),Win Rate,Win Positions,Total Positions,score,rank
0,4020204877254599680,71998.855953,0.311561,0.060748,-0.000923,32.942149,1993,6050,18405.251142,1.0
1,3999240873283311617,42574.473679,0.543673,0.227505,-0.000038,52.321981,2366,4522,10993.443407,2.0
2,4021669203289716224,26427.331592,0.404420,0.068251,-0.000388,31.391201,528,1682,6720.590127,3.0
3,3960874214179953664,19567.471286,0.233296,0.162948,-0.000266,51.568381,1233,2391,5078.323681,4.0
4,3907081197088384000,18015.997370,1.099242,0.222729,-0.001257,61.397148,2540,4137,4844.319362,5.0
5,3956076827719377409,16790.012238,1.032230,0.099528,-0.000390,51.557712,2019,3916,4499.691712,6.0
6,4028701921959171840,17601.401398,0.406480,0.275724,-0.000135,54.294032,373,687,4458.950284,7.0
7,3986814617275053313,16337.461881,0.871628,0.329787,-0.000001,78.221722,2780,3554,4409.187996,8.0
8,3788465932399412480,13960.966457,0.836806,0.152912,-0.001062,44.607364,1369,3069,3716.849621,9.0
9,4022565861939831809,14197.577076,0.233080,0.299602,-0.000063,52.765957,372,705,3608.654026,10.0


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Separate features and target
X = df.drop(columns=['Port','rank'])
y = df['rank']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature columns
scaler =StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the ANN model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32,activation="relu"))
model.add(Dense(16,activation="relu"))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.2)
predictions = model.predict(np.array(X_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')
print(f'Test Loss: {loss}')

c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - loss: 6882.4365 - val_loss: 9586.3428
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 7353.4424 - val_loss: 9523.1260
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 7465.6846 - val_loss: 9435.8252
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 6560.5210 - val_loss: 9308.8438
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 6608.2900 - val_loss: 9119.2422
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 7167.1279 - val_loss: 8828.1963
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 6012.0020 - val_loss: 8395.8350
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 5398.8223 - val_loss: 7772.9888
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5771.6050 - val_loss: 6911.4570
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 4441.2900 - val_loss: 5826.1543
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4030.5945 - val_lo

In [12]:
print(np.concatenate([np.round(predictions),np.array([y_test]).reshape(-1,1)],axis = 1))

[[ 93.  74.]
 [ 21.  19.]
 [106. 119.]
 [ 82.  79.]
 [ 89.  77.]
 [ 28.  32.]
 [ 76.  65.]
 [133. 142.]
 [ 79.  69.]
 [ 93.  83.]
 [108. 111.]
 [  6.  13.]
 [ 42.  37.]
 [  4.  10.]
 [ 20.  20.]
 [ 70.  57.]
 [ 99. 105.]
 [ 63.  70.]
 [ 61.  56.]
 [120. 133.]
 [ 17.  30.]
 [121. 128.]
 [ 23.  27.]
 [120. 129.]
 [121. 132.]
 [146. 146.]
 [101. 109.]
 [141. 144.]
 [ 59.  46.]
 [ 25.  31.]]


In [13]:
lst = os.listdir(r"C:\Users\pc\Desktop\PrimeTrade\Ports")
k = []
for i in lst:
    dict = {}
    Pos = ['SELL SHORT', 'BUY SHORT', 'SELL LONG', 'BUY LONG']
    mydf = pd.read_csv(r"C:\Users\pc\Desktop\PrimeTrade\Ports\{}".format(i))
    mydf["Position"] = mydf["side"] +" " + mydf['positionSide']

    dict["Port"] = i[:-4]
    for j in Pos:
        wining = mydf.query('realizedProfit > 0 and Position == "{}"'.format(j))
        dict["{}".format(j)] = wining.shape[0]
    dict["Total Positions"] = mydf.query('realizedProfit > 0').shape[0]
    k.append(dict)
winning_df = pd.DataFrame(k)

In [14]:
winning_df

,Port,SELL SHORT,BUY SHORT,SELL LONG,BUY LONG,Total Positions
0,3672754654734989568,0,125,85,0,210
1,3733192481840423936,0,0,553,0,553
2,3768170840939476993,0,0,0,0,6
3,3784403294629753856,0,1221,608,0,1829
4,3786761687746711808,0,21,16,0,37
...,...,...,...,...,...,...
145,4039279455324236544,0,0,181,0,181
146,4040382575336130560,0,0,0,0,0
147,4040843843196854529,0,16,3,0,19
148,4041804592937345281,0,81,4,0,85


In [15]:
lst = os.listdir(r"C:\Users\pc\Desktop\PrimeTrade\Ports")
k = []
for i in lst:
    dict = {}
    Pos = ['SELL SHORT', 'BUY SHORT', 'SELL LONG', 'BUY LONG']
    mydf = pd.read_csv(r"C:\Users\pc\Desktop\PrimeTrade\Ports\{}".format(i))
    mydf["Position"] = mydf["side"] +" " + mydf['positionSide']

    dict["Port"] = i[:-4]
    for j in Pos:
        wining = mydf.query('realizedProfit <= 0 and Position == "{}"'.format(j))
        dict["{}".format(j)] = wining.shape[0]
    dict["Total Positions"] = mydf.query('realizedProfit <= 0').shape[0]
    k.append(dict)
lose_df = pd.DataFrame(k)

In [16]:
lose_df

,Port,SELL SHORT,BUY SHORT,SELL LONG,BUY LONG,Total Positions
0,3672754654734989568,77,47,46,94,264
1,3733192481840423936,0,0,7,129,136
2,3768170840939476993,0,0,0,0,8
3,3784403294629753856,2521,251,154,1295,4221
4,3786761687746711808,21,4,0,20,45
...,...,...,...,...,...,...
145,4039279455324236544,0,0,0,146,146
146,4040382575336130560,0,0,0,0,76
147,4040843843196854529,35,0,0,5,40
148,4041804592937345281,77,49,63,94,283
